<a href="https://colab.research.google.com/github/adiltkt/Finetuning-of-LLMs/blob/main/Fine_tunining_of_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

import necessary modules

In [1]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch

# 1. Load dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# 2. Format into conversational prompts
def format_convo(example):
    return {
        "text": f"Context: {example['Context']}\nResponse: {example['Response']}"
    }

formatted_dataset = dataset.map(format_convo)

# 3. Load small GPT-2 model and tokenizer
model_name = "sshleifer/tiny-gpt2"  # ~25MB GPT-2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have pad_token by default

model = AutoModelForCausalLM.from_pretrained(model_name)

# 4. Tokenize the dataset
def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# 5. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="./tiny-gpt2-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    report_to="none",
    fp16=torch.cuda.is_available()
)

# 7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 8. Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

/tmp/ipython-input-1-3960084089.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,10.737800
20,10.740000
30,10.736000
40,10.738100
50,10.738600
60,10.735300
70,10.733600
80,10.731600
90,10.730400
100,10.727000


TrainOutput(global_step=2634, training_loss=10.576266741336251, metrics={'train_runtime': 1279.4597, 'train_samples_per_second': 8.235, 'train_steps_per_second': 2.059, 'total_flos': 1229930496.0, 'train_loss': 10.576266741336251, 'epoch': 3.0})

In [38]:
def normalize(text):
    return text.lower().strip()


Evaluation metrics for the model

In [10]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [18]:
pip install evaluate

In [39]:


import evaluate

# Load common metrics
bleu_metric = evaluate.load("bleu")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")


In [40]:
import evaluate
from tqdm import tqdm

# Load metrics
bleu = evaluate.load("bleu")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

# Generate predictions
predictions = []
references = []

model.eval()
for example in tqdm(dataset["train"]):  # Replace with your dataset split
    input_text = f"Context: {example['Context']}\nResponse:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded_pred = tokenizer.decode(output[0], skip_special_tokens=True).split("Response:")[-1].strip()
    decoded_ref = example["Response"].strip()

    predictions.append(decoded_pred)
    references.append([decoded_ref])  # BLEU requires a list of references

# Compute metrics
bleu_result = bleu.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_result['bleu']:.4f}")



100%|██████████| 3512/3512 [07:48<00:00,  7.49it/s]


BLEU: 0.0000


In [43]:
precision = precision_metric.compute(predictions=pred_labels, references=true_labels)
print(f"Precision: {precision['precision']:.4f}")

Precision: 0.6667


Evaluation usecases

In [29]:
# Assume we have lists of predictions and references (tokenized or raw text depending on the metric)
predictions = ["The cat sat on the mat", "He is reading a book"]
references = [["The cat is on the mat"], ["He is reading a book"]]

# BLEU
bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['bleu']:.4f}")

BLEU: 0.6340


In [33]:

# F1 / Precision / Recall (example with labels)
true_labels = [1, 0, 1, 1, 0]
pred_labels = [1, 0, 0, 1, 1]
f1 = f1_metric.compute(predictions=pred_labels, references=true_labels, average="macro")
precision = precision_metric.compute(predictions=pred_labels, references=true_labels, average="macro")
recall = recall_metric.compute(predictions=pred_labels, references=true_labels, average="macro")

print(f"F1 Score: {f1['f1']:.4f}")
print(f"Precision: {precision['precision']:.4f}")
print(f"Recall: {recall['recall']:.4f}")


F1 Score: 0.5833
Precision: 0.5833
Recall: 0.5833


Generate a Prompt

In [35]:
prompt = "Context: I have a headache.\nResponse:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2,
    eos_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: I have a headache.
Response: always doesnResponse wants along or "  me with my is would some you do not and that at girl. other way of am stop perhaps what can needs factors my you or meResponse mutual I The because to me it your even through makes as you


In [41]:
from transformers import set_seed
import torch

prompt = "Context: I cannot sleep.\nResponse:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Optional: Set seed for reproducibility
set_seed(42)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: I cannot sleep.
Response: but for factors some you perhaps can about to person not for about my There with that do, wants way. doesn what people asked like but so having always your along
 are you The more of I girl through It because are it andResponse about even
